# Create Harmonization Benchmark To GDC Data Dictionary

This notebook utilizes data from the following paper:

* Yurong Liu, Eduardo H. M. Pena, Aécio Santos, Eden Wu, and Juliana Freire. 2025. Magneto: Combining Small and Large Language Models for Schema Matching. Proc. VLDB Endow. 18, 8 (April 2025), 2681–2694. https://doi.org/10.14778/3742728.3742757

## Setup

In [ ]:
# If you are actively working on related *.py files and would like changes to reload automatically into this notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json
import glob
import copy
import csv

## Get benchmark data

Set input and output directories

In [ ]:
input_dir = os.path.abspath(
    "../datasets/harmonization_benchmark_real_GDC/inputs"
)

output_dir = os.path.abspath(
    "../datasets/harmonization_benchmark_real_GDC/outputs"
)

Get GDC Data Dictionary as a target model

In [ ]:
url = "https://api.gdc.cancer.gov/v0/submission/_dictionary/_all"
target_model_path = os.path.abspath(input_dir) + "/target_model_GDC.json"
os.makedirs(os.path.dirname(target_model_path), exist_ok=True)
!wget -q -O "{target_model_path}" "{url}"

Get 10 source CSVs from the paper as a source tables

In [ ]:
source_tables = [
    ("1MyQOryVm3S0iBz3-uqAC_bPqZjMtS6IA", "Cao.csv"), # pragma: allowlist secret
    ("1N3rbTHtnVDe19kMNei0opy_g-8Hr_Hl5", "Clark.csv"), # pragma: allowlist secret
    ("1Ml-lY2LnAwpFpgHGeE7R2qqWRxBVLso9", "Dou.csv"), # pragma: allowlist secret
    ("1Nac7mZR_reZPdK5zghI5Y3pEKq8VPTRQ", "Gilette.csv"), # pragma: allowlist secret
    ("1NIFT5dHcguZ1vzbQ_qz1tIhNDx-QENSe", "Huang.csv"), # pragma: allowlist secret
    ("1MjNgXn-peUUaSadqIcWlqszECgxw7-ux", "Krug.csv"), # pragma: allowlist secret
    ("1ND-qu_62kGtz98O23AMFId4SHQX5GPzJ", "McDermott.csv"), # pragma: allowlist secret
    ("1NE13PtlXR6w2wRXyZrY6ar2lUXeLUw1-", "Satpathy.csv"), # pragma: allowlist secret
    ("1MxEwZbz-31bQqM8ECIKnrClSQNTNwwpY", "Vasaikar.csv"), # pragma: allowlist secret
    ("1NgEsOT7jPdCll0Q3iQ_tuAMqe8L2XFBE", "Wang.csv") # pragma: allowlist secret
]

source_tables_path = os.path.abspath(input_dir) + "/source_tables"
os.makedirs(source_tables_path, exist_ok=True)


for id, name in source_tables:
    url = f"https://drive.google.com/uc?export=download&id={id}"
    !wget -q --no-check-certificate "{url}" -O "{source_tables_path}/{name}"

Get 10 ground truth mappings CSVs from the paper as source mappings

In [ ]:
source_mappings = [
    ("1c64T1cq09T6WmOIIMGRO6yglIRBDDaYP", "Cao.csv"), # pragma: allowlist secret
    ("10pzRiZWuhE_jfNAm7D8XzKzM7ebJgbyj", "Clark.csv"), # pragma: allowlist secret
    ("1vqL5HhFT6SxptQu7FyLidJnn2VKb4UMg", "Dou.csv"), # pragma: allowlist secret
    ("1S0Fe2YlcqNhO1aFMwnePLjKVm1LPVDL8", "Gilette.csv"), # pragma: allowlist secret
    ("1Jy3FIE8jcrNiNlyXsoQIo86nfGVSeAsL", "Huang.csv"), # pragma: allowlist secret
    ("1VS27jhKjNjxPnxn4SJt3OcvbMItYSfG2", "Krug.csv"), # pragma: allowlist secret
    ("107WFZ_-kCY-Yh9MGn1Fx1N93b23be27D", "McDermott.csv"), # pragma: allowlist secret
    ("1JY5fo4Tg3b_bgp-6JHPqweCiunjpWqPe", "Satpathy.csv"), # pragma: allowlist secret
    ("1qZ_kOz9-iC8IzMSvdRHhZIc-mjrU-aSk", "Vasaikar.csv"), # pragma: allowlist secret
    ("1N8h2qwWBy8IO7QMx9ahkUE6vuhDdT6El", "Wang.csv") # pragma: allowlist secret
]

source_mappings_path = os.path.abspath(input_dir) + "/source_mappings"
os.makedirs(source_mappings_path, exist_ok=True)


for id, name in source_mappings:
    url = f"https://drive.google.com/uc?export=download&id={id}"
    !wget -q --no-check-certificate "{url}" -O "{source_mappings_path}/{name}"

## Format benchmark data

Some columns contain muliple numeric values seperated by semicolon (ex "1;2;3"), this function will extract numeric values from these strings

In [ ]:
def extract_numeric_from_mixed(col_values):
    flat = []
    for val in col_values.dropna():
        elements = str(val).split(";")
        for x in elements:
            try:
                num = float(x)
                flat.append(num)
            except ValueError:
                continue
    return pd.Series(flat, dtype=float)

This function creates JSONS from CSV

In [ ]:
def csv_to_schema(df, bins=5, thresh_numeric=0.5, file_name=None):
    schema = {"type": "object", "properties": []}
    if file_name:
        basename = os.path.splitext(file_name)[0]
        schema["name"] = basename
    for col in df.columns:
        col_info = {}
        col_values = df[col]
        numeric = pd.to_numeric(col_values, errors='coerce')
        col_info['name'] = col
        if numeric.notnull().sum() >= thresh_numeric * col_values.notnull().sum():
            numeric_values = extract_numeric_from_mixed(col_values)
            numeric_values = numeric_values.dropna().astype(float)
            if len(numeric_values) > 1:
                counts, bin_edges = np.histogram(numeric_values, bins=bins)
                bin_mids = ((bin_edges[:-1] + bin_edges[1:]) / 2).tolist()
            else:
                bin_mids = numeric_values.tolist()
                counts = [1] * len(numeric_values)
            col_info['type'] = 'number'
            col_info['histogram'] = {
                "bins": bin_mids,
                "counts": counts if isinstance(counts, list) else counts.tolist()
            }
        else:
            value_list = []
            for val in col_values.dropna():
                if ';' in str(val):
                    value_list.extend(str(val).split(';'))
                elif '|' in str(val):
                    value_list.extend(str(val).split('|'))
                else:
                    value_list.append(val)
            counts = pd.Series(value_list).value_counts()
            bins_ = counts.index.tolist()
            values = counts.values.tolist()
            col_info['type'] = 'string'
            col_info['histogram'] = {
                "bins": bins_,
                "counts": values
            }
        schema["properties"].append(col_info)
    return schema

Convert source CSVs to JSONs models and save source and target models in {source}_{target} folder

In [ ]:
target_name = "GDC"

for fname in os.listdir(source_tables_path):
    source_name = os.path.splitext(fname)[0]
    source_target_path = f"{os.path.abspath(output_dir)}/{source_name}_{target_name}"
    os.makedirs(source_target_path, exist_ok=True)
    if fname.endswith(".csv"):
        csv_path = os.path.join(source_tables_path, fname)
        try:
            df = pd.read_csv(csv_path)
            schema = csv_to_schema(df, bins=5, file_name=fname)
            json_path = os.path.join(source_target_path, f"source_model.json")
            with open(json_path, "w") as f:
                json.dump(schema, f, indent=2)
            print(f"Source files processed and saved: {json_path}")
            target_path = shutil.copy(target_model_path, source_target_path + "/target_model.json")
            print(f"Target model copied to: {target_path}")
        except Exception as e:
            print(f"Error processing {fname}: {e}")

Format mappings and save them in {source}_{target} folder

In [ ]:
# Load previously downloaded GDC dictionary
with open(target_model_path, "r") as f:
    gdc_dict = json.load(f)

attribute_to_node = {}

# Iterate all top-level keys (skip ones starting with "_", e.g. "_definitions")
for node_name, node in gdc_dict.items():
    if node_name.startswith("_"):
        continue              # these are not real data nodes
    if "properties" in node:
        for prop in node["properties"].keys():
            attribute_to_node[prop] = node_name

# For debugging
print("Sample mapping:", dict(list(attribute_to_node.items())[:10]))

for fname in os.listdir(source_mappings_path):
    if fname.lower().endswith(".csv"):
        source_name = os.path.splitext(fname)[0]
        source_target_path = f"{os.path.abspath(output_dir)}/{source_name}_{target_name}"
        os.makedirs(source_target_path, exist_ok=True)
        df = pd.read_csv(os.path.join(source_mappings_path, fname))
        src_col = [c for c in df.columns if "original" in c.lower()][0]
        trg_col = [c for c in df.columns if "gdc" in c.lower()][0]
        rows = []
        for _, row in df.iterrows():
            src = f"{source_name}.{str(row[src_col]).strip()}"
            trg_var = str(row[trg_col]).strip()
            node_prefix = attribute_to_node.get(trg_var, "")
            trg = f"{node_prefix}.{trg_var}" if node_prefix else trg_var
            rows.append({"source_node_prop_type_desc": src, "target_node_prop_type_desc": trg})
        out_df = pd.DataFrame(rows)
        out_tsv = os.path.join(source_target_path, f"expected_mappings.tsv")
        out_df.to_csv(out_tsv, index=False, sep='\t')
        print(f"Mappings processed and saved: {out_tsv}")

Format of output:

- output_dir
    - source_target_folder_0
        - `source_model.json`
        - `expected_mappings.tsv`
        - `target_model.json`
    - ...

## Construct Single Benchmark Test File

Code derived from harmonization_real_benchmark_creation.ipynb

Each test should include a source model, with desire to harmonize to a target. We expect harmonization `expected_mappings.tsv`.

Now let's create a JSONL file with a test per row.

The JSONL file should have 3 columns: `input_source_model`, `input_target_model`, `harmonized_mapping`

Those 3 columns should be populated by content of the files:

- `source_model.json` == `input_source_model`
- `expected_mappings.tsv` == `harmonized_mapping`
- `target_model.json` == `input_target_model`

In [ ]:
def create_jsonl_from_structure(root_dir, output_jsonl_path):
    """
    Iterates through subfolders under root_dir and writes a single JSONL file
    with input_source_model, input_target_model, harmonized_mapping fields.
    """
    records = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # find the first source_model.json file in this directory
        print(f"Current dir: {dirpath}")
        print(f"Files in dir: {filenames}")
        source_model_files = glob.glob(os.path.join(dirpath, "source_model*"))
        expected_mappings_path = os.path.join(dirpath, "expected_mappings.tsv")
        target_model_files = glob.glob(os.path.join(dirpath, "target_model*"))
        if (
            source_model_files
            and os.path.isfile(expected_mappings_path)
            and target_model_files
        ):
            source_model_path = source_model_files[0]
            target_model_path = target_model_files[0]
            # Read files
            with open(source_model_path, "r", encoding="utf-8") as input_file:
                input_source_model = json.load(input_file)
            with open(expected_mappings_path, "r", encoding="utf-8") as input_file:
                harmonized_mapping = input_file.read()
            with open(target_model_path, "r", encoding="utf-8") as input_file:
                input_target_model = json.load(input_file)
            record = {
                "input_source_model": input_source_model,
                "input_target_model": input_target_model,
                "harmonized_mapping": harmonized_mapping,
            }
            records.append(record)

    print(f"Test count: {len(records)}")
    with open(output_jsonl_path, "w", encoding="utf-8") as output_file:
        for record in records:
            output_file.write(json.dumps(record) + "\n")

In [ ]:
output_json_filepath = os.path.join(output_dir, "output.jsonl")
create_jsonl_from_structure(output_dir, output_json_filepath)

In [ ]:
def harmonization_data_jsonl_to_csv(jsonl_file, csv_file, input_headers=None):
    """
    Converts a JSONL file to a CSV file.

    Headers must include: `harmonized_mapping`

    This denormalizes the harmonized mapping so each property mapped is its own row.
    """
    input_headers = input_headers or [
        "input_source_model",
        "input_target_model",
        "harmonized_mapping",
    ]

    if "harmonized_mapping" not in input_headers:
        raise Exception("Headers must include: `harmonized_mapping`")

    input_headers.remove("harmonized_mapping")
    output_headers = copy.deepcopy(input_headers)
    output_headers.extend(
        [
            "source_node_prop_type_desc",
            "target_node_prop_type_desc",
        ]
    )

    with open(jsonl_file, "r") as f_in, open(csv_file, "w", newline="") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(output_headers)

        for line in f_in:
            if not line.strip():
                continue
            try:
                data = json.loads(line)
                if not data:
                    continue
                for single_property_harmonized_mapping in data[
                    "harmonized_mapping"
                ].split("\n")[1:]:
                    if not single_property_harmonized_mapping:
                        continue
                    source_node_prop_type_desc, target_node_prop_type_desc = (
                        single_property_harmonized_mapping.split("\t")
                    )
                    row = []
                    for header in input_headers:
                        if header == "harmonized_mapping":
                            continue
                        row.append(data[header])
                    row += [
                        source_node_prop_type_desc,
                        target_node_prop_type_desc,
                    ]
                    writer.writerow(row)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {line.strip()} - {e}")

In [ ]:
harmonization_data_jsonl_to_csv(
    f"{output_dir}/output.jsonl",
    f"{output_dir}/output.csv",
)

## Validate Test File

Code derived from harmonization_real_benchmark_creation.ipynb

In [ ]:
import io
import csv
from harmonization.simple_data_model import (
    SimpleDataModel,
    get_node_prop_type_desc_from_string,
)

benchmark_filepath = f"{output_dir}/output.jsonl"

# since these files are separated by target model already, just get the first row
input_target_model = ""
with open(benchmark_filepath, "r", encoding="utf-8") as infile:
    for line in infile:
        row = json.loads(line)
        try:
            input_target_model = json.loads(row["input_target_model"])
        except Exception:
            input_target_model = row["input_target_model"]

        try:
            input_source_model = json.loads(row["input_source_model"])
        except Exception:
            input_source_model = row["input_source_model"]

        target_model = SimpleDataModel.get_from_unknown_json_format(
            json.dumps(input_target_model)
        )
        target_model_props_lookup = {}
        for node in target_model.nodes:
            for node_property in node.properties:
                target_model_props_lookup[
                    f"{node.name}.{node_property.name}".strip()
                ] = node_property

        source_model = SimpleDataModel.from_simple_json(json.dumps(input_source_model))
        source_model_props_lookup = {}
        for node in source_model.nodes:
            for node_property in node.properties:
                source_model_props_lookup[
                    f"{node.name}.{node_property.name}".strip()
                ] = node_property

        print("Checking that all target props actually exist in target model...")

        harmonized_mapping = row["harmonized_mapping"]

        # Use io.StringIO to treat the string as a file-like object
        tsv_file = io.StringIO(harmonized_mapping)
        reader = csv.reader(tsv_file, delimiter="\t")

        # Skip the header row
        next(reader)

        for node_prop_mapping in reader:
            source_model_node_prop_type_desc, target_model_node_prop_type_desc = (
                node_prop_mapping
            )

            source_node_name, source_prop_name, source_prop_type, source_prop_desc = (
                get_node_prop_type_desc_from_string(source_model_node_prop_type_desc)
            )
            (
                target_node_name,
                target_prop_name,
                target_prop_type,
                target_prop_desc,
            ) = get_node_prop_type_desc_from_string(target_model_node_prop_type_desc)

            if (
                f"{target_node_name}.{target_prop_name}".strip()
                not in target_model_props_lookup
            ):
                print(
                    f"ERROR: {target_node_name}.{target_prop_name} is not in target_model"
                )
                pass

            if (
                f"{source_node_name}.{source_prop_name}".strip()
                not in source_model_props_lookup
            ):
                print(
                    f"ERROR: {source_node_name}.{source_prop_name} is not in source_model"
                )